In [53]:
import pandas as pd
import numpy as np
from scipy import optimize
import numpy_financial as npf
import itertools 

#Ponemos la información de entrada

#Datos de los captadores

A=1.9     #Área de un colector solar
N= 6      #Número de captadores
sc= A*N   #Área predimensionada


#Datos del colector
n_1= 0.696 #Eficiencia óptica del colector
n_2= 0.94  #Modificador del ángulo de incidencia
n_3= 0.95 #Factor de corrección del conjunto intercambiador
U= 4.821   #Coeficiente de pérdidas lineales

F_R_t= n_1*n_2*n_3
F_R_U_l= n_3*U


#Producción de CO2
CO2_ratio= 0.41    #CO2equi/kWh

#Costos
sol_ratio= 0.77   #soles/kWh
cos_sc= 320          #soles/m2
cos_V= 0.72          #soles/kg

#Importamos datos mensuales de la temperatura de agua de red, radiación media diaria por unidad de superficie, Factor de corrección sobre superficie inclinada
t_mensual= pd.read_csv("input_month.csv", sep=";")

f_value=[]
aporte_solar_MJ=[]
rendimiento=[]
backup_SOL=[]
CO2_kg=[]
ahorro=[]
number_collector=[]
tir=[]
inversion= []
VAN=[]
payback=[]
opt_cost=pd.DataFrame(columns=["N_colectors","income",'outcome',"cash_flow"])


interest_rate = 0.1  # The default guess value

#Cálculo para cada una de las áreas 

for N in range (1,21):
    #Calculations
    sc= A*N   #Área predimensionada (m2)
    V= A*75   #Volumen predimensionado (l)
    invertion= sc*cos_sc+V*cos_V
    t_mensual["X"]=F_R_U_l*3600*(100-t_mensual["Temp_outdoor"])*t_mensual["Sun_hours"]*t_mensual["Days_month"]*sc/(t_mensual["Thermal_demand_MJ"]*1000000)
    t_mensual["Y"]= n_1*n_3*n_2*t_mensual["Rad_enegy_MJ"]*t_mensual["Days_month"]*1000000*sc/(t_mensual["Thermal_demand_MJ"]*1000000)
    t_mensual["f_value"]=1.020*t_mensual["Y"]-0.065*t_mensual["X"]-0.245*t_mensual["Y"].pow(2)+ 0.0018*t_mensual["X"]+0.0215*t_mensual["Y"].pow(3)
    t_mensual["aporte_solar_MJ"]= t_mensual["f_value"]*t_mensual["Thermal_demand_MJ"]
    t_mensual["rendimiento"]= t_mensual["aporte_solar_MJ"]/(t_mensual["Rad_enegy_MJ"]*t_mensual["Days_month"]*sc)
    t_mensual["backup_cost_SOL"]= (t_mensual["Thermal_demand_MJ"]-t_mensual["aporte_solar_MJ"])*sol_ratio*0.2777
    t_mensual["CO2_kg"]= (t_mensual["Thermal_demand_MJ"]-t_mensual["aporte_solar_MJ"])*CO2_ratio
    t_mensual["Ahorro"]=(t_mensual["Thermal_demand_MJ"]-t_mensual["aporte_solar_MJ"])*sol_ratio*0.277
    
    #Important parameters results
    f_value.append(t_mensual["aporte_solar_MJ"].sum()/t_mensual["Thermal_demand_MJ"].sum())
    rendimiento.append(t_mensual["rendimiento"].mean())
    aporte_solar_MJ.append(t_mensual.aporte_solar_MJ.sum())
    number_collector.append(N)
    backup_SOL.append(t_mensual["backup_cost_SOL"].sum())
    CO2_kg.append(t_mensual["CO2_kg"].sum())
    inversion.append(invertion)
    

    #Calculations of the cash flows
    
    income=[]
    outcome=[]
    years=[]
    N_col=[]
    
    for a in range (1,6):
        outcome.append(t_mensual["backup_cost_SOL"].sum())
        income.append((t_mensual["Thermal_demand_MJ"]*0.277*sol_ratio).sum())
        years.append(a)
        N_col.append(N)
 
    balance= pd.DataFrame({'income':income,'outcome':outcome, "years":years, "N_colectors":N_col})
    balance["cash_flow"]= balance["income"]-balance["outcome"]
    
    cash= [invertion*-1]
    cash.append(list(balance["cash_flow"]))
    flat_list = itertools.chain(*cash)
    flat_list=[]
        
    def flatten_list (data):
    
        for element in data:
            if type(element) == list:
                flatten_list(element)
            else:
                flat_list.append(element)
            
    flatten_list(cash)
    
    VAN_i= npf.npv(interest_rate,flat_list)
    VAN.append(VAN_i)
    
    tir_i= npf.irr(flat_list)
    tir.append(tir_i)
    
    
    cumulative_cash_flows = 0
    payback_period= 0
    
    while cumulative_cash_flows < invertion:
        cumulative_cash_flows += flat_list [payback_period]
        payback_period+=1
        
        if payback_period== len(flat_list):
            break
    payback.append(payback_period)
df_technical_results=pd.DataFrame(list(zip(number_collector,rendimiento, f_value,aporte_solar_MJ, backup_SOL, CO2_kg,inversion, VAN, tir, payback)), columns=["number_collector","rendimiento","f_value","aporte_solar_MJ", "backup_SOL", "CO2_kg", "inversion", "VAN", "TIR", "payback"])
df_technical_results.to_csv("CleanData/tech_results_df.csv")
details_of_costs= opt_cost.to_csv("CleanData/detail_costs_df.csv")

    

In [54]:
df_technical_results

,number_collector,rendimiento,f_value,aporte_solar_MJ,backup_SOL,CO2_kg,inversion,VAN,TIR,payback
0,1,0.559442,0.113345,6514.308839,10896.540846,20893.245288,710.6,4452.337700,1.907422,3
1,2,0.541460,0.218454,12555.318044,9604.797889,18416.431514,1318.6,8741.059811,2.004300,2
2,3,0.523937,0.315678,18143.073773,8409.973669,16125.451666,1926.6,12662.383655,1.989210,3
3,4,0.506874,0.405363,23297.622185,7307.781736,14012.086817,2534.6,16232.558250,1.944433,3
4,5,0.490272,0.487860,28039.009440,6293.935641,12068.118042,3142.6,19467.832614,1.888536,3
5,6,0.474129,0.563517,32387.281695,5364.148933,10285.326417,3750.6,22384.455766,1.828041,3
6,7,0.458446,0.632683,36362.485111,4514.135162,8655.493017,4358.6,24998.676723,1.765826,3
7,8,0.443223,0.695707,39984.665846,3739.607877,7170.398916,4966.6,27326.744506,1.703359,3
8,9,0.428460,0.752937,43273.870059,3036.280630,5821.825188,5574.6,29384.908131,1.641466,3
9,10,0.414157,0.804722,46250.143909,2399.866969,4601.552910,6182.6,31189.416617,1.580647,3
